<a href="https://colab.research.google.com/github/hey0wing/Parallel_TedTalk/blob/main/Parallel_TedTalk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import os,glob
import time
!pip3 install requests
import requests
import http

In [2]:
# create /talks/ if do not exist
if not os.path.exists('talks'):
    os.makedirs('talks')

# clear the pwd when needed
def delCSV ():
  filenames = glob.glob(os.getcwd() + "/*.csv")
  for filename in filenames:
    os.remove(filename)

delCSV ()
os.chdir('talks')
delCSV ()
os.chdir('/content/')

# Extract talks names

Avoid HTTP status 429 of TedTalk, request again after sleep time

In [3]:
sleep = 0.5
def requestTed(path):
  r = requests.get(path)
  while r.status_code == 429:
    # sleep to wait
    time.sleep(sleep)
    r = requestTed(path)
  return r

In [4]:
# EXTRACT TED Talk Names
def talk_count(TargetPath, TargetLang):
  dict_ = {}
  for lang in TargetLang:
    main = TargetPath + lang
    r = requestTed(main)
    soup = BeautifulSoup(r.text)
    page = soup.find_all("a",class_='pagination__item pagination__link')
    max = 0
    for p in page:
      p = int(p.text)
      if p > max:
        max = p
    dict_[lang] = max
    print('Talks in', lang, ':', max)
  return min(dict_, key=dict_.get)

In [5]:
# EXTRACT TED Talk Names
def enlist_talk_names(TargetPath, lang):
  dict_ = {}
  main = TargetPath + lang
  i = 0
  while 1 != 0:
    i += 1
    path = main + '&page=' + str(i)
    r = requestTed(path)
    soup = BeautifulSoup(r.text)
    talks = soup.find_all("a",class_='')
    if soup.find(class_='browse__no-results'):
      break
    print (i, path)

    for t in talks:
      href = t.attrs['href'][:-(10+len(lang))]
      if href.find('/talks/') == 0 and dict_.get(href) != 1:
        # print(href)
        dict_[href]=1

  return dict_

Choose targeted parallel languages

In [6]:
TargetLang = ['zh', 'en']
TargetPath = 'https://www.ted.com/talks?language='

# noise = [' (Applause) ', ' （掌聲） ', ' (Laughter) ', '（笑聲） ', ' (Guitar) ', ' （吉他）']
noise = []
less_talk = talk_count(TargetPath, TargetLang)
print('Less talks:',less_talk)
all_talk_names = enlist_talk_names(TargetPath, less_talk)
print()
print('Total number of talks:', (len(all_talk_names)))

Talks in zh : 6
Talks in en : 121
Less talks: zh
1 https://www.ted.com/talks?language=zh&page=1
2 https://www.ted.com/talks?language=zh&page=2
3 https://www.ted.com/talks?language=zh&page=3
4 https://www.ted.com/talks?language=zh&page=4
5 https://www.ted.com/talks?language=zh&page=5
6 https://www.ted.com/talks?language=zh&page=6

Total number of talks: 193


# Extract subtitles

In [7]:
import re
def extract_talk(name, lang):
  main = 'https://www.ted.com' + name
  print(main)
  all = []
  parallel_short = {}
  parallel_long = {}
  for i in lang:
    path = main + '/transcript?/transcript&language=' + i
    # print(path)
    r = requestTed(path)
    soup = BeautifulSoup(r.text)
    # if failed, try again
    while len(soup.findAll('p')) == 0:
      r = requestTed(path)
      soup = BeautifulSoup(r.text)

    text_talk = []
    for x in soup.findAll('p'):
      x = [x for x in re.split(r'[\n\t]+', x.text) if x]
      if x not in noise:
        text_talk.append(x)
    text_talk = text_talk[:len(text_talk)-2]
    all.append(text_talk)
    parallel_short[i] = len(all)-1
    parallel_long[i] = len(all)-1

  short1 = []
  short2 = []
  long1 = []
  long2 = []
  if len(all[0]) == len(all[1]) and len(all[0]) != 0:
    for i in range(len(all[0])):
      if len(all[0][i]) == len(all[1][i]):
        short1.extend(all[0][i])
        short2.extend(all[1][i])
      else:
        long1.append(' '.join(all[0][i]))
        long2.append(' '.join(all[1][i]))
    short = [short1, short2]
    long = [long1, long2]

    for k in parallel_short.keys():
      parallel_short[k] = short[parallel_short[k]]
      parallel_long[k] = long[parallel_long[k]]

    p_s_length = [len(parallel_short[x]) for x in TargetLang]
    p_l_length = [len(parallel_long[x]) for x in TargetLang]
    if p_s_length[0] != 0:
      # for x in TargetLang:
        # print(parallel_short[x][0])
      df_s = pd.DataFrame(parallel_short)
      df_s.to_csv(os.getcwd() + name + '_short.csv', sep='\t', encoding='utf-8')
      if p_l_length[0] != 0:
        df_l = pd.DataFrame(parallel_long)
        df_l.to_csv(os.getcwd() + name + '_long.csv', sep='\t', encoding='utf-8')
        return 1, p_s_length[0], 1, p_l_length[0]
      else:
        return 1, p_s_length[0], 0, 0

    else:
      if p_l_length[0] != 0:
        df_l = pd.DataFrame(parallel_long)
        df_l.to_csv(os.getcwd() + name + '_long.csv', sep='\t', encoding='utf-8')
        # print('Only long corpus available\n')
        return 0, 0, 1, p_l_length[0]
      else:
        # print('Cannot retreive subtitle for both')
        # print(p_s_length[0], p_s_length[1])
        return 0, 0, 0, 0
    
  else:
    # print('Not parallel corpus, different length')
    # print(len(all[0]), len(all[1]))
    return 0, 0, 0, 0

In [8]:
Total_count = 0
count_short = 0
corpus_length_short = 0
count_long = 0
corpus_length_long = 0
start = time.time()

for i in all_talk_names:
  c_short, l_short, c_long, l_long = extract_talk(i, TargetLang)
  count_short += c_short
  corpus_length_short += l_short
  count_long += c_long
  corpus_length_long += l_long
  Total_count += 1
  if Total_count%10 == 0:
    # print('- '*10)
    print('Total Count:', Total_count)
    print('Short Count:', count_short)
    print('Short Corpus length:', corpus_length_short)
    print('Long Count:', count_long)
    print('Long Corpus length:', corpus_length_long)
    print('Running time:', time.time()-start)
    print('- '*10)
end = time.time()

https://www.ted.com/talks/kristen_bell_giant_ant_why_act_now
https://www.ted.com/talks/brent_loken_can_we_create_the_perfect_farm
https://www.ted.com/talks/matt_walker_how_sleep_can_improve_your_immunity
https://www.ted.com/talks/matt_walker_how_sleep_affects_your_emotions
https://www.ted.com/talks/matt_walker_how_caffeine_and_alcohol_affect_your_sleep
https://www.ted.com/talks/shekinah_elmore_the_courage_to_live_with_radical_uncertainty
https://www.ted.com/talks/aparna_pallavi_what_foods_did_your_ancestors_love
https://www.ted.com/talks/elizabeth_cox_what_is_a_coronavirus
https://www.ted.com/talks/anees_bahji_what_is_schizophrenia
https://www.ted.com/talks/jim_hudspeth_the_beautiful_mysterious_science_of_how_you_hear
Total Count: 10
Short Count: 7
Short Corpus length: 49
Long Count: 8
Long Corpus length: 80
Running time: 8.313566446304321
- - - - - - - - - - 
https://www.ted.com/talks/iseult_gillespie_the_tale_of_the_doctor_who_defied_death
https://www.ted.com/talks/kristen_wenz_what_

In [9]:
print('Number of short Talks: ' + str(count_short))
print('Length of short corpus: ' + str(corpus_length_short))
print()
print('Number of long Talks: ' + str(count_long))
print('Length of long corpus: ' + str(corpus_length_long))
print('- '*20, '\nTime: ' + str(end - start))

Number of short Talks: 119
Length of short corpus: 7657

Number of long Talks: 104
Length of long corpus: 1404
- - - - - - - - - - - - - - - - - - - -  
Time: 628.5544934272766


# Output CSV files

In [10]:
short_pieces = []
long_pieces = []
if os.getcwd() != '/content/talks':
  os.chdir('talks')

for file in glob.glob('*_short.csv'):
  frame = pd.read_csv(os.getcwd() + '/' + file, sep='\t', encoding='utf-8')
  short_pieces.append(frame)
df_short = pd.concat(short_pieces, ignore_index=True)

for file in glob.glob('*_long.csv'):
  frame = pd.read_csv(os.getcwd() + '/' + file, sep='\t', encoding='utf-8')
  long_pieces.append(frame)
df_long = pd.concat(long_pieces, ignore_index=True)

In [11]:
os.chdir('/content/')
df_short.to_csv('All_TED_TALKS_DATA_SHORT.csv',sep='\t',encoding='utf-8')
df_long.to_csv('All_TED_TALKS_DATA_LONG.csv',sep='\t',encoding='utf-8')

In [12]:
df_short

,Unnamed: 0,zh,en
0,0,大家好，我叫 Frank,"Hi, my name is Frank,"
1,1,我收集人哋嘅秘密,and I collect secrets.
2,2,而一切都源於我,It all started with a crazy idea
3,3,2004 年 11 月一個瘋狂嘅諗法,in November of 2004.
4,4,我當時打印咗三千張,"I printed up 3,000 self-addressed postcards,"
...,...,...,...
7652,25,咁你哋個女都會一樣,"your daughters would be, too."
7653,26,請你哋對經期抱著積極的態度,So please be period positive.
7654,27,（笑聲）,(Laughter)
7655,28,多謝,Thank you.


In [13]:
df_long

,Unnamed: 0,zh,en
0,0,今時今日，有四千萬個美國人 孭住一筆讀書債投身社會 佢哋屋企窮，要借錢讀大學 而家夾埋嘅債務...,Today 40 million Americans are indebted for th...
1,1,以前响美國 讀大學唔等於要負債纍纍咁畢業 我朋友阿 Paul 佢爸爸喺科羅拉多州立大學畢業 ...,"Once upon a time in America, going to college ..."
2,2,阿 Kate 同埋其他大學生都要借錢 因為大學學費經已貴到 好多美國家庭都負擔唔起 咁又點啫...,Students like Kate have to take on a loan beca...
3,3,所以… 學費飆升 津貼少咗 家庭收入又縮咗水 個人收入微薄 唔怪得有多過四分一嘅畢業生 連要...,"So ... tuition costs up, public funding down, ..."
4,4,1.2 兆美金嘅學債 明顯反映咗 大學教育經已淪為一種消費品 我哋講起教育就好似班資本家咁 ...,1.2 trillion dollars of debts for diplomas mak...
...,...,...,...
1399,16,如果呢部火車係不需要停靠呢？ 當你喺路上嘅時候 呢部火車嘅車廂可以動態分離 變成快車、無人駕...,What if in this train you didn't have to stop?...
1400,17,宜家…… 就好似喺城市行走，有無人機 空中的士、組合式巴士同埋磁懸浮車艙 都仲未夠神奇 我覺...,Now ... as if walking in a city buzzing with d...
1401,18,但係當成個城市變成 無人駕駛嘅時候會點？ 我哋仲需唔需要紅綠燈？ 仲需唔需要車道？ 速度限制...,But what would happen when whole cities become...
1402,19,交通功能將會非常豐富 會好似血液咁流動 根據一個奇怪嘅悖論 交通網絡越係自動化 佢嘅流動就越...,Traffic will be functionally exuberant. It wil...
